In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio openai pyunsplash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
  Created wheel for pyunsplash: filename=pyunsplash-1.0.0rc2-py3-none-any.whl size=12968 sha256=8d17deda41a076b8f56d799ffba067ffb48179ce4be79c48b03bcba3fc048967
  Stored in directory: /root/.cache/pip/wheels/89/f7/7d/878ade22be264d33306f00c0cd0451ad

In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import uvicorn

In [ ]:
# @title Demo
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
from fastapi.responses import JSONResponse, FileResponse
from typing import Optional
# Patch the event loop
nest_asyncio.apply()

app = FastAPI()

@app.get("/status")
async def status_check():
    return JSONResponse(content={"status": "API is running"})

@app.get("/recommend_images")
async def recommend_images(query: Optional[str] = Query(None, description="Query string for image recommendation")):
    # Based on the query, recommend an image. Here, we're just returning a static file as an example.
    image_path = "/content/image1.png"  # Replace with logic to select the appropriate image
    return FileResponse(image_path, media_type="image/png")

@app.get("/recommend_video")
async def recommend_video(query: Optional[str] = Query(None, description="Query string for video recommendation")):
    # Based on the query, recommend a video. Here, we're just returning a static file as an example.
    video_path = "/content/video1.mp4"  # Replace with logic to select the appropriate video
    return FileResponse(video_path, media_type="video/mp4")

public_url = ngrok.connect(8000)
print(public_url)
# Run the Uvicorn server
uvicorn.run(app, host="0.0.0.0", port=8000)



# Logic Development

In [ ]:
# @title Keyword Extraction
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=("APIKEYHERE")
def analyze_text_for_keywords(content: str):
    """
    Analyzes the provided text to extract keywords using the OpenAI API.
    """
    client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get('OPENAI_KEY'),
    )
    prompt = f"Analyze the following text and generate a list of keywords: {content}"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a keyword extraction assistant,  Understand the context and its supposed to be used to find sutable images, add relevant additional keywords based on the scenario which can be used while searching the image. Only return sutable text that is needed to search the image, make sure to highlight the main object of the context, return only string of search"},
            {"role": "user", "content": prompt}
        ]
    )
    keywords = response.choices[0].message
    keywords = keywords.content

    return keywords

In [ ]:
api_response = analyze_text_for_keywords("Hi this is a very big bug")
print(api_response)

big bug, insect, critter, macro photography, wildlife, nature, pests


In [ ]:
# @title  AI Image Prompt Generator
def generate_ai_image_prompt(content: str):
    """
    Generates a detailed prompt for AI image generation based on the text content.
    """
    prompt = f"Create a detailed and specific image generation prompt for the following content: {content}"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a AI Image Prompt Generator. Provide Prompts to generate image in DALLE using Photo Realistic Images with real world lighting , be very technical with the camera settings, it should not feel like an AI generated image, Will pass the content generate the prompt accordingly,  must be under 200 tokens in total , add negative prompts to remove unwanted artifacts use 100 tokens for positive pompt and 100 for  negative prompt to remove artifacting, only return just the prompt and no other text"},
            {"role": "user", "content": prompt}
        ]
    )
    imageprompt = response.choices[0].message
    imageprompt = imageprompt.content
    return imageprompt

In [ ]:
PROMPT = generate_ai_image_prompt("There was a little boy who loved to come and play around the tree every day. He used to climb to the treetop, eat the apples, and take naps under its shade. He loved the tree and the tree also loved to play with him. As time went by, the little boy grew up and he would no longer play around the tree every day.")

In [ ]:
PROMPT = """**Positive Prompt:** A young boy playing joyfully around a large, lush green tree in a sunlit park. The boy is climbing to the treetop, his face beaming with happiness, surrounded by vibrant red apples hanging from the branches. In the foreground, soft sunlight filters through the leaves, creating dappled shadows on the ground. The scene captures the essence of childhood joy and friendship with nature. Camera settings: 50mm lens, f/2.8 aperture, ISO 100, 1/200s shutter speed for sharp details and soft background blur.

**Negative Prompt:** Remove any unnatural textures, stark shadows, pixelation, or distortion in the tree and boy. Exclude any harsh lighting, overly saturated colors, or visible digital artifacts. Avoid unrealistic proportions or perspectives, and ensure no ghosting effects or unwanted lens flares in the scene."""

In [ ]:
# @title AI Image Generation
from openai import OpenAI
from google.colab import userdata
def generate_ai_image(PROMPT: str):
    """
    Generates images using the OpenAI API based on the provided prompt.
    """
    client = OpenAI(
        # This is the default and can be omitted
        api_key=userdata.get('OPENAI_KEY'),
        )
    response = client.images.generate(
      model="dall-e-3",
      prompt=PROMPT,
      size="1024x1024",
      n=1,
    )

    url = response.data[0].url
    return url


In [ ]:
print(generate_ai_image(PROMPT))

https://oaidalleapiprodscus.blob.core.windows.net/private/org-vA4Uor6dRcDGllOLlqOvuUEH/user-4jRL43H2e9BoqaoOH9DVjiGX/img-9Fqo6BTP4EJ5Ubxt6VaTSnSG.png?st=2024-08-24T07%3A55%3A13Z&se=2024-08-24T09%3A55%3A13Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-08-23T23%3A30%3A00Z&ske=2024-08-24T23%3A30%3A00Z&sks=b&skv=2024-08-04&sig=NUuEAXyt%2Bpz9t82y1e/UIqtq6cbAHzsanSbNNgd78AU%3D


In [ ]:
# @title Unsplashed API
from pyunsplash import PyUnsplash
unsplash_api_key = userdata.get('unsplash_api_key')

pu = PyUnsplash(api_key=unsplash_api_key)

def fetch_unsplash_images(keywords: str):
    """
    Fetches images from Unsplash based on the provided keywords using PyUnsplash.
    """
    #loop through the list and get the images
    search = pu.search(type_="photos", query=keywords, per_page=1)
    images = [photo.link_download for photo in search.entries]
    return images

In [ ]:
keywords = analyze_text_for_keywords(PROMPT)

fetch_unsplash_images(keywords)

['https://unsplash.com/photos/wtIPKwwYGOU/download?ixid=M3w2NDM0Nzl8MHwxfHNlYXJjaHwxfHwqKktleXdvcmRzJTNBKiolMEEtJTIwWW91bmclMjBib3klMEEtJTIwSm95ZnVsJTIwcGxheSUwQS0lMjBMdXNoJTIwZ3JlZW4lMjB0cmVlJTBBLSUyMFN1bmxpdCUyMHBhcmslMEEtJTIwQ2xpbWJpbmclMjB0cmVldG9wJTBBLSUyMEhhcHBpbmVzcyUwQS0lMjBWaWJyYW50JTIwcmVkJTIwYXBwbGVzJTBBLSUyMFNvZnQlMjBzdW5saWdodCUwQS0lMjBEYXBwbGVkJTIwc2hhZG93cyUwQS0lMjBDaGlsZGhvb2QlMjBqb3klMEEtJTIwRnJpZW5kc2hpcCUyMHdpdGglMjBuYXR1cmUlMEEtJTIwQ2FtZXJhJTIwc2V0dGluZ3MlMEEtJTIwNTBtbSUyMGxlbnMlMEEtJTIwZiUyRjIuOCUyMGFwZXJ0dXJlJTBBLSUyMElTTyUyMDEwMCUwQS0lMjAxJTJGMjAwcyUyMHNodXR0ZXIlMjBzcGVlZCUwQS0lMjBTaGFycCUyMGRldGFpbHMlMEEtJTIwU29mdCUyMGJhY2tncm91bmQlMjBibHVyJTBBJTBBKipBZGRpdGlvbmFsJTIwS2V5d29yZHMlM0EqKiUwQS0lMjBPdXRkb29yJTIwYWR2ZW50dXJlJTBBLSUyME5hdHVyZSUwQS0lMjBGYW1pbHklMEEtJTIwU2NlbmljJTIwYmVhdXR5JTBBLSUyMFBsYXlmdWwlMjBpbm5vY2VuY2UlMEEtJTIwTGlnaHQlMjBmaWx0ZXJpbmclMjB0aHJvdWdoJTIwbGVhdmVzJTBBLSUyMENoaWxkcmVuJTIwaW4lMjBuYXR1cmUlMEEtJTIwR3JlZW5lcnklMEEtJTIwRnJ1aXQlMjB0cmVlcyUwQS0l

In [ ]:
keywords

'**Keywords:**\n- Young boy\n- Joyful play\n- Lush green tree\n- Sunlit park\n- Climbing treetop\n- Happiness\n- Vibrant red apples\n- Soft sunlight\n- Dappled shadows\n- Childhood joy\n- Friendship with nature\n- Camera settings\n- 50mm lens\n- f/2.8 aperture\n- ISO 100\n- 1/200s shutter speed\n- Sharp details\n- Soft background blur\n\n**Additional Keywords:**\n- Outdoor adventure\n- Nature\n- Family\n- Scenic beauty\n- Playful innocence\n- Light filtering through leaves\n- Children in nature\n- Greenery\n- Fruit trees\n- Sunlight effects\n\n**Search Sentence:**\n"Young boy joyfully climbing a lush green tree with vibrant red apples in a sunlit park, capturing childhood joy and nature\'s beauty."'

# Final Implementation

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
from fastapi.responses import JSONResponse, FileResponse
from typing import Optional
import openai
from pyunsplash import PyUnsplash
from pyngrok import ngrok
import openai

ngrok_authToken = userdata.get('NGROCK')
ngrok.set_auth_token(ngrok_authToken)

# Patch the event loop
nest_asyncio.apply()

app = FastAPI()
client = OpenAI(
# This is the default and can be omitted
api_key=userdata.get('OPENAI_KEY'),
)
# Set your OpenAI AP
unsplash_api_key = userdata.get('unsplash_api_key')

pu = PyUnsplash(api_key=unsplash_api_key)

def analyze_text_for_keywords(content: str):
    """
    Analyzes the provided text to extract keywords using the OpenAI API.
    """
    client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get('OPENAI_KEY'),
    )
    prompt = f"Analyze the following text and generate a list of keywords: {content}"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a keyword extraction assistant,  Understand the context and its supposed to be used to find sutable images, add relevant additional keywords based on the scenario which can be used while searching the image. Write a sentence of the search term"},
            {"role": "user", "content": prompt}
        ]
    )
    keywords = response.choices[0].message
    keywords = keywords.content

    return keywords

def generate_ai_image_prompt(content: str):
    """
    Generates a detailed prompt for AI image generation based on the text content.
    """
    prompt = f"Create a detailed and specific image generation prompt for the following content: {content}"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a AI Image Prompt Generator. Provide Prompts to generate image in DALLE using Photo Realistic Images with real world lighting , ,  must be under 200 tokens in total only return just the prompt and no other text"},
            {"role": "user", "content": prompt}
        ]
    )
    imageprompt = response.choices[0].message
    imageprompt = imageprompt.content
    return imageprompt

def generate_ai_image(PROMPT: str):
    """
    Generates images using the OpenAI API based on the provided prompt.
    """
    client = OpenAI(
        # This is the default and can be omitted
        api_key=userdata.get('OPENAI_KEY'),
        )
    response = client.images.generate(
        prompt=PROMPT,
        n=2,
        size="512x512"
    )
    url = response.data[0].url
    return url

def fetch_unsplash_images(keywords: str):
    """
    Fetches images from Unsplash based on the provided keywords using PyUnsplash.
    """
    #loop through the list and get the images
    search = pu.search(type_="photos", query=keywords, per_page=1)
    images = [photo.link_download for photo in search.entries]
    return images[0]

@app.get("/status")
async def status_check():
    return JSONResponse(content={"status": "API is running"})

@app.get("/recommend_images")
async def recommend_images(query: Optional[str] = Query(None, description="Query string for image recommendation"),
                           use_ai: Optional[bool] = Query(False, description="Use AI to generate images")):
    if query:
      if use_ai:
            PROMPT = generate_ai_image_prompt(query)
            IMAGE_URL = generate_ai_image(PROMPT)
            #return JSONResponse(content={"url": IMAGE_URL})
            return JSONResponse(IMAGE_URL)
      else:
            keywords = analyze_text_for_keywords(query)
            images = fetch_unsplash_images(keywords)
            return JSONResponse(images)
            # return JSONResponse(content={"keywords": keywords, "images": images})
    else:
        image_path = "/content/image1.png"  # Replace with logic to select the appropriate image
        return FileResponse(image_path, media_type="image/png")



public_url = ngrok.connect(8000)
print(public_url)

# Run the Uvicorn server
uvicorn.run(app, host="0.0.0.0", port=8000)


NgrokTunnel: "https://1c27-35-237-199-106.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [420]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /recommend_images?query=Chicago+police+ignore+warnings+about+press+freedom+at+DNC+protests%0AWe+warned+them%2C+in+print+and+on+the+radio%2C+that+dispersing+law-abiding+journalists+violates+the+First+Amendment.+They+did+it+anyway&use_ai=True HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /status HTTP/1.1" 200 OK
INFO:     117.219.57.84:0 - "GET /recommend_images?query=Chicago+police+ignore+warnings+about+press+freedom+at+DNC+protests%0AWe+warned+them%2C+in+print+and+on+the+radio%2C+th

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [420]
